In [20]:
import requests as rq
import bs4 as bs
import re
import time

In [15]:
url = f"https://www.atlasobscura.com/places?page=202&sort=likes_count"
raw = rq.get(url)
soup = bs.BeautifulSoup(raw.content)

In [18]:
raw.status_code == 200

True

In [9]:
len(soup.find_all("a",attrs={'data-place-id':re.compile(r"\d*")}))

0

In [25]:
cities_url = []
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
for page in range(1,401):
    try:
        print(page)
        url = f"https://www.atlasobscura.com/places?page={page}&sort=likes_count"
        raw = rq.get(url,headers=headers)
        if raw.status_code != 200:
            print("Sleeping")
            time.sleep(120)
            raw = rq.get(url,headers=headers)
        
        soup = bs.BeautifulSoup(raw.content)
        
        for d in soup.find_all("a",attrs={'data-place-id':re.compile(r"\d+")}):
            cities_url.append(d["href"])
            
    except Exception as e:
        print(page)
        print(e)
        print(raw.status_code)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [26]:
len(cities_url)

7200

In [28]:
with open("links.txt","w") as f:
    for link in cities_url:
        complete_url = "https://www.atlasobscura.com"+link+"\n"
        f.write(complete_url)

In [19]:
url = f"https://www.atlasobscura.com/places?page=2&sort=likes_count"

In [20]:
raw = rq.get(url)
soup = bs.BeautifulSoup(raw.content)

In [22]:
with open("pretty.txt","w") as f:
    f.write(soup.prettify())